In [1]:
# script to save the rankings for the mechanism
import os
import sys
import copy
import pickle
import pandas as pd

import numpy as np
import rmgpy.chemkin
import cantera as ct

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/dft')
import importlib
import thermokinetic_fun
importlib.reload(thermokinetic_fun)

<module 'thermokinetic_fun' from '/work/westgroup/harris.se/autoscience/reaction_calculator/dft/thermokinetic_fun.py'>

## Load the model for reaction and species descriptions

In [2]:
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week'

cantera_file = os.path.join(basedir, 'chem_annotated.cti')
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)

gas = ct.Solution(cantera_file)
perturbed_cti_path = os.path.join(basedir, 'perturbed.cti')
perturbed_gas = ct.Solution(perturbed_cti_path)

# This cti -> rmg converter dictionary can be made using rmg_tools/ct2rmg_dict.py
with open(os.path.join(basedir, 'ct2rmg_rxn.pickle'), 'rb') as handle:
    ct2rmg_rxn = pickle.load(handle)

print(f'{len(species_list)} species loaded')
print(f'{len(reaction_list)} reactions loaded')

130 species loaded
2488 reactions loaded


In [3]:
N = len(gas.species())
M = len(gas.reactions())

In [4]:
# check for outliers that get crushed by average

In [5]:
M

2523

## Gather Uncertainty Rankings

In [6]:
# rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/reaction_uncertainty.npy'
# sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/species_uncertainty.npy'

rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_reaction_uncertainty.npy'
sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_species_uncertainty.npy'

rmg_rxn_uncertainty = np.load(rxn_uncertainty_file)
rmg_sp_uncertainty = np.load(sp_uncertainty_file)

assert len(rmg_rxn_uncertainty) == len(reaction_list)
assert len(rmg_sp_uncertainty) == len(species_list)


rxn_uncertainty = np.zeros(len(gas.reactions()))
for ct_index in range(len(rxn_uncertainty)):
    rxn_uncertainty[ct_index] = rmg_rxn_uncertainty[ct2rmg_rxn[ct_index]]

# Cantera species should be in same rmg order, but this makes sure for us
for i in range(len(species_list)):
    assert str(species_list[i]) == gas.species_names[i]

sp_uncertainty = rmg_sp_uncertainty

total_uncertainty_array = np.concatenate((sp_uncertainty, rxn_uncertainty), axis=0)
total_uncertainty_mat = np.repeat(np.transpose(np.matrix(total_uncertainty_array)), 12 * 51, axis=1)


In [7]:

# Display the most uncertain parameters

In [8]:
# Calculate Improvement Score
DFT_error = 3.0
# improvement_score = np.abs(reaction_sensitivities) * (uncorrelated_uncertainties - DFT_error)
# improvement_score_order = [x for _,x in sorted(zip(improvement_score, reaction_indices))][::-1]
reaction_indices = np.arange(0, len(gas.reactions()))
reaction_uncertainty_order = [x for _,x in sorted(zip(rxn_uncertainty, reaction_indices))][::-1]

In [9]:
print('Top Uncertain Reactions')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 10):
    ct_index = reaction_uncertainty_order[i]
    print(ct_index, '\t', np.round(rxn_uncertainty[ct_index], 3),
          '\t', gas.reactions()[ct_index], 
          '\t', reaction_list[ct2rmg_rxn[ct_index]].family)
#           '\t', f'{reaction_sensitivities[ct_index]:.3e}',
#           '\t', f'{improvement_score[ct_index]:.3e}')
    

Top Uncertain Reactions
i	Delta	Reaction	Sensitivity	Improvement Score
2174 	 28.541 	 C3H6(688) + CH3(18) <=> C3H5-A(94) + CH4(10) 	 Disproportionation
2168 	 28.541 	 C3H6(688) + CH2(23) <=> C3H5-A(94) + CH3(18) 	 Disproportionation
1251 	 28.541 	 C3H5O(129) + H(14) <=> C3H4O(74) + H2(13) 	 Disproportionation
1248 	 28.541 	 C3H5O(129) + CH3(18) <=> C3H4O(74) + CH4(10) 	 Disproportionation
1242 	 28.541 	 C3H5O(129) + CH2(23) <=> C3H4O(74) + CH3(18) 	 Disproportionation
1001 	 28.541 	 C2H5O(49) + CH2(23) <=> CH3(18) + CH3CHO(35) 	 Disproportionation
1000 	 28.541 	 C2H5O(49) + CH3(18) <=> CH3CHO(35) + CH4(10) 	 Disproportionation
958 	 28.541 	 CH2(23) + S(777) <=> CH3(18) + S(252) 	 Disproportionation
955 	 28.541 	 H(14) + S(777) <=> H2(13) + S(252) 	 Disproportionation
953 	 28.541 	 CH3(18) + S(777) <=> CH4(10) + S(252) 	 Disproportionation


## Gather Sensitivity Rankings

In [10]:
# load the giant base delays matrix
base_delay_file = os.path.join(basedir, 'total_base_delays.npy')
base_delays = np.load(base_delay_file)

# Load the giant delays matrix
total_delay_file = os.path.join(basedir, 'total_perturbed_mech_delays.npy')
total_delays = np.load(total_delay_file)

assert total_delays.shape[1] == len(base_delays)

total_base_delays = np.repeat(np.matrix(base_delays), total_delays.shape[0], axis=0)
total_base_delays[total_base_delays == 0] = np.nan
assert total_base_delays.shape == total_delays.shape

total_delays[total_delays == 0] = np.nan


In [11]:
d_ln_tau = np.log(total_delays) - np.log(total_base_delays)

In [12]:
avg_d_ln_tau = np.nanmean(d_ln_tau, axis = 1)
avg_d_ln_tau[np.isnan(avg_d_ln_tau)] = -np.inf

/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [13]:
d_ln_tau

matrix([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-7.32677030e-10, -5.38507627e-08, -4.39428050e-10, ...,
          2.11526707e-07,  6.30630181e-07, -3.27123875e-08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]])

### Get $\Delta G$ or $\Delta \ln k$ for each parameter

In [14]:
# get Delta G from the simulation gas

In [15]:
phi_dicts = []

for table_index in range(1, 13):
    
    # Load the experimental conditions
    ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    # ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    df_exp = pd.read_csv(ignition_delay_data)
    table_exp = df_exp[df_exp['Table'] == table_index]
    # Define Initial conditions using experimental data
    tau_exp = table_exp['time (ms)'].values.astype(float)  # ignition delay
    T7 = table_exp['T_C'].values  # Temperatures
    P7 = table_exp['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
    phi7 = table_exp['phi'].values  # equivalence ratios
    # list of starting conditions
    # Mixture compositions taken from table 2 of
    # https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
    concentrations = []
    # for phi = 1
    x_diluent = 0.7649
    conc_dict = {
        'O2(2)': 0.2038,
        'butane(1)': 0.03135
    }

    
    x_N2 = table_exp['%N2'].values[0] / 100.0 * x_diluent
    x_Ar = table_exp['%Ar'].values[0] / 100.0 * x_diluent
    x_CO2 = table_exp['%CO2'].values[0] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    
    phi_dicts.append(conc_dict)
        
        

In [16]:
# There are 12 * K different simulation settings. We need each parameter estimate at each setting
# Create a matrix with temperatures and one with pressures
T = np.linspace(663, 1077, 51)
table_temperatures = np.repeat(np.matrix(T), 12, axis=1)
temperatures = np.repeat(table_temperatures, total_delays.shape[0], axis=0)

pressures = np.zeros(temperatures.shape)
for i in range(pressures.shape[1]):
    if int(i / 51) in [0, 3, 6, 9]:
        pressures[:, i] = 10.0 * 101325.0
    elif int(i / 51) in [1, 4, 7, 10]:
        pressures[:, i] = 20.0 * 101325.0
    elif int(i / 51) in [2, 5, 8, 11]:
        pressures[:, i] = 30.0 * 101325.0

In [17]:
G_base = np.zeros((N, total_delays.shape[1]))
G_perturbed = np.zeros((N, total_delays.shape[1]))


# get base G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_base[j, i] = gas.species()[j].thermo.h(T) - T * gas.species()[j].thermo.s(T)


In [18]:
# Get perturned G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
#     print(j)
    # change just the one reaction
    mod_gas.modify_species(j, perturbed_gas.species()[j])
    
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        mod_gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_perturbed[j, i] = mod_gas.species()[j].thermo.h(T) - T * mod_gas.species()[j].thermo.s(T)

    mod_gas.modify_species(j, gas.species()[j])

In [19]:
# In theory, delta G should be 10% G_base, but apparently it isn't...
# G has units Enthalpy [J/kg or J/kmol] it's J / kmol
delta_G = G_perturbed - G_base
delta_G_kcal_mol = delta_G / 4.184 / 1000.0 / 1000.0  # needs to be kcal/mol to match Gao paper

In [20]:
np.min(delta_G_kcal_mol - (G_base * 0.1 / 4.184 / 1000.0 / 1000.0))

1.7499742014181026

### Now do reaction rate

In [21]:
# except we know that by definition, this is 0.1

In [22]:
delta_ln_k = 0.1 * np.ones((M, total_delays.shape[1]))

In [23]:
# concatenate into a big delta matrix
delta = np.concatenate((delta_G_kcal_mol, delta_ln_k), axis=0)

### Put it all together into $\frac{\partial \ln \tau}{\partial G}$ or $\frac{\partial \ln \tau}{\partial \ln k}$

In [24]:
# first derivative is change in delay / change in G
first_derivative = np.divide(d_ln_tau, delta)

### Display most sensitive parameters

In [25]:
avg_first_derivative = np.nanmean(first_derivative, axis=1)
avg_first_derivative[np.isnan(avg_first_derivative)] = -np.inf

parameter_indices = np.arange(0, N + M)
reaction_sensitivity_order = [x for _, x in sorted(zip(avg_first_derivative, parameter_indices))][::-1]

print('Top Sensitive Parameters')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 10):
    ct_index = reaction_sensitivity_order[i]
    
    if ct_index < N:
        print(ct_index, '\t', np.round(avg_first_derivative[ct_index,0], 9),
              '\t', gas.species()[ct_index], )
    else:
        print(ct_index, '\t', np.round(avg_first_derivative[ct_index,0], 9),
              '\t', gas.reactions()[ct_index - N])

Top Sensitive Parameters
i	Delta	Reaction	Sensitivity	Improvement Score
398 	 0.472593429 	 OH(15) + butane(1) <=> H2O(8) + SC4H9(183)
190 	 0.290840547 	 2 HO2(16) <=> H2O2(17) + O2(2)
150 	 0.290840537 	 2 HO2(16) <=> H2O2(17) + O2(2)
430 	 0.285117774 	 S(186) <=> C4H8(189) + HO2(16)
81 	 0.232724916 	 <Species S(223)>
468 	 0.197582231 	 S(184) <=> C4H8(188) + HO2(16)
469 	 0.191072452 	 S(186) <=> C4H8(188) + HO2(16)
603 	 0.150505413 	 S(219) <=> C4H8O(214) + OH(15)
294 	 0.054769036 	 CH3(18) + HO2(16) <=> CH4(10) + O2(2)
268 	 0.05436085 	 CH2O(9) + OH(15) <=> H2O(8) + HCO(19)


/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


# Compute Improvement Score

In [26]:
delta_uncertainty_squared = total_uncertainty_mat - 3.0  # assume this is already squared in Connie's calculations
sensitivity_squared = np.float_power(first_derivative, 2.0)

improvement_score = np.multiply(delta_uncertainty_squared, sensitivity_squared)

avg_improvement_score = np.nanmean(improvement_score, axis=1)
avg_improvement_score[np.isnan(avg_improvement_score)] = -np.inf

improvement_score[np.isnan(improvement_score)] = -np.inf

/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


In [27]:
len(avg_improvement_score)

2653

### Display Top Improvement Scores

In [34]:
parameter_indices = np.arange(0, N + M)
improvement_order = [x for _, x in sorted(zip(avg_improvement_score, parameter_indices))][::-1]

print('Top Improvement Scores')
print('i\tCt Index\tImprovement Score\tReaction')
new_top50 = set()
for i in range(0, 55):
    ct_index = improvement_order[i]
    
    
    if ct_index < N:
        print(i, '\t', ct_index, '\t', np.round(avg_improvement_score[ct_index, 0], 9),
              '\t', gas.species()[ct_index], )
        new_top50.add(ct_index)
    else:
        family = 'PDEP'
        try:
            family = reaction_list[ct2rmg_rxn[ct_index - N]].family
        except AttributeError:
            pass
        print(i, '\t', ct_index - N, '\t', np.round(avg_improvement_score[ct_index, 0], 9),
              '\t', gas.reactions()[ct_index - N], family)
        new_top50.add(ct_index - N)

Top Improvement Scores
i	Ct Index	Improvement Score	Reaction
0 	 959 	 0.01302262 	 OH(15) + S(777) <=> S(787) R_Recombination
1 	 232 	 0.000155586 	 C2H5(33) + HO2(16) <=> C2H4(11) + H2O2(17) Disproportionation
2 	 943 	 0.000121797 	 O2(2) + S(777) <=> HO2(16) + S(252) Disproportionation
3 	 2469 	 4.0034e-05 	 C2H3CO(75) + C3H6(12) <=> C3H4O(74) + C3H5-A(94) H_Abstraction
4 	 776 	 2.2236e-05 	 C4H8(189) + HCO(19) <=> C4H7(190) + CH2O(9) H_Abstraction
5 	 821 	 2.1321e-05 	 C4H7(190) + HO2(16) <=> C4H6(194) + H2O2(17) Disproportionation
6 	 2283 	 1.5324e-05 	 C2H3OH(58) + HCO(19) <=> CH2CHO(21) + CH2O(9) H_Abstraction
7 	 346 	 1.1804e-05 	 S(229) <=> S(225) intra_H_migration
8 	 446 	 7.226e-06 	 HO2(16) + SC4H9(183) <=> C4H8(188) + H2O2(17) Disproportionation
9 	 99 	 6.281e-06 	 <Species S(911)>
10 	 443 	 6.214e-06 	 CH2CHO(21) + HO2(16) <=> CH2CO(24) + H2O2(17) Disproportionation
11 	 1693 	 4.681e-06 	 C2H5O2(47) + CH3O(31) <=> C2H6O2(48) + CH2O(9) Disproportionation
12 	 17

In [31]:
new_top50 - set(old_top50)

{1144, 1806, 2471}

In [30]:
old_top50 = [
    959,
232,
943,
2469,
776,
821,
2283,
346,
99,
443,
446,
1764,
1693,
777,
231,
752,
742,
1315,
1842,
483,
102,
2346,
1139,
1789,
751,
828,
1711,
605,
822,
757,
1316,
2352,
1138,
434,
1794,
754,
1131,
606,
393,
1749,
2472,
1686,
871,
394,
1713,
1126,
614,
1550,
823,
2356,
444,
861,

]

In [ ]:
# get just the top 50 database reaction numbers
my_improvement_index = 991
my_improvement_index = 574
my_improvement_index = 1027
my_improvement_index = 1936
my_improvement_index = 2601
my_improvement_index = 1484
my_improvement_index = 739
my_improvement_index = 742

my_improvement_index = 1683

rmg_index = ct2rmg_rxn[my_improvement_index - N]
rxn_smiles = thermokinetic_fun.reaction2smiles(reaction_list[rmg_index])
db_index = thermokinetic_fun.reaction_smiles2index(rxn_smiles)
print(db_index)


In [36]:
# get just the top 50 database reaction numbers
db_indices = np.zeros(N + M)
for i in range(0, 55):
    ct_index = improvement_order[i]
    if ct_index < N:
        continue
    rmg_index = ct2rmg_rxn[ct_index - N]
    rxn_smiles = thermokinetic_fun.reaction2smiles(reaction_list[rmg_index])
    db_index = thermokinetic_fun.reaction_smiles2index(rxn_smiles)
    db_indices[ct_index] = db_index
    print(rmg_index, db_index)

931 931
213 213
915 915
2434 2422
748 748
793 793
2248 2240
324 324
422 422
419 419
1665 1665
1736 1736
749 749
212 212
724 724
1814 1814
714 714
1287 1287
459 459
800 800
2311 2303
1111 1111
723 723
1761 1761
1683 1683
577 577
1110 1110
794 794
728 728
1288 1288
2317 2309
410 410
726 726
1103 1103
2437 2425
1766 1766
578 578
1721 1721
370 370
843 843
1658 1658
833 833
1685 1685
1522 1522
795 795
1098 1098
371 371
586 586
2321 2313
420 420
1778 1778
2436 2424
1116 1116


In [37]:
# save the results to a csv
total_df = pd.DataFrame()

# db_indices = []
ct_indices = []
rmg_indices = []
names = []
entry_types = []
sensitivities = []
uncertainties = []
improvement_scores = []

for i in range(0, len(gas.species())):
    smiles = species_list[i].smiles
    db_indices[i] = thermokinetic_fun.species_smiles2index(smiles)
#     db_indices.append(thermokinetic_fun.species_smiles2index(smiles))
    ct_indices.append(i)
    rmg_indices.append(i)
    names.append(smiles)
    entry_types.append('species')
    sensitivities.append(avg_first_derivative[i, 0])
    uncertainties.append(total_uncertainty_array[i])
    improvement_scores.append(avg_improvement_score[i, 0])
    

for i in range(0, len(gas.reactions())):
    rmg_index = ct2rmg_rxn[i]
#     rxn_smiles = thermokinetic_fun.reaction2smiles(reaction_list[rmg_index])
#     db_index = thermokinetic_fun.reaction_smiles2index(rxn_smiles)
#     db_indices.append(db_index)
    ct_indices.append(i)
    rmg_indices.append(rmg_index)
    names.append(str(reaction_list[rmg_index]))
    entry_types.append('reaction')
    sensitivities.append(avg_first_derivative[N + i, 0])
    uncertainties.append(total_uncertainty_array[N + i])
    improvement_scores.append(avg_improvement_score[N + i, 0])
    

total_df['db_index'] = db_indices
total_df['ct_index'] = ct_indices
total_df['rmg_index'] = rmg_indices
total_df['name'] = names
total_df['entry_type'] = entry_types
total_df['sensitivity'] = sensitivities
total_df['uncertainty'] = uncertainties
total_df['improvement_score'] = improvement_scores
total_df.to_csv(os.path.join(basedir, 'mechanism_summary.csv'))


In [ ]:
i = 1110
print(reaction_list[i])
print(reaction_list[i].family)

In [ ]:
param_index = 1003
these_delays = np.array(first_derivative[param_index,:])
plt.plot(np.transpose(these_delays))
# print(these_delays.shape)

print(avg_first_derivative[param_index])

In [ ]:
print(first_derivative[param_index, 156])
print(temperatures[param_index, 156])
print(pressures[param_index, 156])
print(phi_dicts[int(156 / 51)])

In [ ]:
np.argmax(these_delays)

In [ ]:
these_delays[0, 415]

In [ ]:
len(phi_dicts)

In [ ]:
delta[0, :]

In [ ]:
first_derivative[0, 415]

In [ ]:
my_slice = total_delays[:, 156] - np.ravel(total_base_delays[:, 156])[:]

plt.plot(my_slice)

In [ ]:
np.nanmax(my_slice)

In [ ]:
for i in range(0, 200):
    if my_slice[i] == np.nanmax(my_slice):
        print(i, my_slice[i])

In [ ]:
my_slice[97]

In [ ]:
my_slice.shape

In [ ]:
total_delays[:, 156].shape

In [ ]:
i = 0
plt.plot(np.ravel(first_derivative[i, :]))

In [ ]:
first_derivative[0, 156]

## Get Improvement Score by Condition

In [ ]:
# get improvement score at a particular condition:

condition_index = 382  # all top 10 are in top 50

condition_index = 415
# nothing at condition 415 worked

In [ ]:
improvement_score.shape

In [ ]:
single_improvement_score = np.ravel(improvement_score[:, condition_index])
single_improvement_score[np.isnan(single_improvement_score)] = -np.inf

In [ ]:
parameter_indices = np.arange(0, N + M)
single_improvement_order = [x for _, x in sorted(zip(single_improvement_score, parameter_indices))][::-1]

print('Top Improvement Scores')
print('i\tDelta\tReaction\tSensitivity\tImprovement Score')
for i in range(0, 10):
    ct_index = single_improvement_order[i]
    
    if ct_index < N:
        print(ct_index, '\t', np.round(single_improvement_score[ct_index], 9),
              '\t', gas.species()[ct_index], )
    else:
        family = 'PDEP'
        try:
            family = reaction_list[ct2rmg_rxn[ct_index - N]].family
        except AttributeError:
            pass
        print(ct_index - N, '\t', np.round(single_improvement_score[ct_index], 9),
              '\t', gas.reactions()[ct_index - N], family)

In [ ]:
top20 = set()

# check that all top 10 in the single condition scenario are in the top 50 for the average
for condition_index in range(improvement_score.shape[1]):
    single_improvement_score = np.ravel(improvement_score[:, condition_index])
    single_improvement_score[np.isnan(single_improvement_score)] = -np.inf

    parameter_indices = np.arange(0, N + M)
    single_improvement_order = [x for _, x in sorted(zip(single_improvement_score, parameter_indices))][::-1]

#     print(condition_index)
    if condition_index == 415:
        continue
    for i in range(0, 20):
        ct_index = single_improvement_order[i]
        if ct_index not in improvement_order[0:50]:
            top20.add(ct_index)
            print(condition_index, ':\t', ct_index, 'mising!!!!!!!!!!!!!!!!!!')

    


In [ ]:
print(top20)

In [ ]:
len(top20)

In [ ]:
# condition 415. says we should include 2643-2652

In [ ]:
improvement_score[2643, 415]

In [ ]:
improvement_score[:, 415]

In [ ]:
rxns2calc = set()
improvement_threshold = 2e-7
for i in range(N + M):
    if np.any(np.ravel(improvement_score[i, :]) > improvement_threshold):
        rxns2calc.add(i)
    
print(rxns2calc)

In [ ]:
for i in rxns2calc:
    if i not in improvement_order[0:50]:
        print(i)

In [ ]:
print(len(rxns2calc))

In [ ]:
improvement

In [ ]:
plt.plot(np.ravel(improvement_score[991,:]))